# GloVe

https://www.youtube.com/watch?v=RyTpzZQrHCs

1. Word vector space have substructure
2. Design algorithms with that structure in mind
3. Older models are often better than you think

<img src='cooccurTable.png'>

Log-bilinear model: $w_i \cdot w_j = \log P(i|j)$

vector differences: $w_x \cdot (w_a-w_b) = \log \frac{P(x|a)}{P(x|b)}$

think: a='ice' and b='steam'


# Code


In [ ]:
def build_cooccur(vocab, corpus, window_size=10, min_count=None):
    vocab_size = len(vocab)
    id2word = dict((i, word) for word, (i, _) in vocab.iteritems())

    # Collect cooccurrences internally as a sparse matrix for
    # passable indexing speed; we'll convert into a list later
    cooccurrences = sparse.lil_matrix((vocab_size, vocab_size),
                                      dtype=np.float64)
# -- continued --
    for i, line in enumerate(corpus):
        tokens = line.strip().split()
        token_ids = [vocab[word][0] for word in tokens]
        
# -- continued --
        for center_i, center_id in enumerate(token_ids):
            # Collect all word IDs in left window of center word
            context_ids = token_ids[max(0, center_i - window_size)
                                    : center_i]
            contexts_len = len(context_ids)
# -- continued --
            for left_i, left_id in enumerate(context_ids):
                # Distance from center word
                distance = contexts_len - left_i

                # Weight by inverse of distance between words
                increment = 1.0 / float(distance)

                # Build co-occurrence matrix symmetrically (pretend
                # we are calculating right contexts as well)
                cooccurrences[center_id, left_id] += increment
                cooccurrences[left_id, center_id] += increment

def train_glove(vocab, cooccurrences, vector_size=100,
                iterations=25, **kwargs):
